In [26]:
import dgl
import pandas as pd
import torch
import numpy as np

In [27]:
import sys
sys.path.append("..")
from dev.policy import *

In [28]:
p = "pool_0001"
s = "schedule_0002"

pool, schedule = pd.read_csv(f'../dev/pools/{p}.csv',dtype={'employee_id':'str'}), \
                 pd.read_csv(f'../dev/schedules/{s}.csv',dtype={'shift_id':'str'})


shift_features, count_workers = 8,2

In [29]:
schedule = pd.get_dummies(schedule,drop_first=True)
shift_features = schedule.shape[1]
for i in pd.get_dummies(pool).columns.to_list():
    schedule[i] = 0

schedule = schedule
state = schedule.to_numpy()
state

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0]])

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(shift_features, count_workers, 32, 32, 32)
decoder = Decoder()
policy = Policy(encoder, decoder).to(device)

In [31]:
bg, shift_index = policy.grapher(state)
bg

Graph(num_nodes=8, num_edges=24,
      ndata_schemes={'x': Scheme(shape=(32,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})

In [32]:
encoded_graph = encoder(bg, bg.ndata['x'])

In [33]:
encoded_graph.ndata['h'][4:]

tensor([[-0.4178, -0.4862,  0.2360, -0.0494, -0.2569, -0.9166,  0.0618, -0.3302,
          0.0510,  0.3893,  0.2635,  0.3080, -0.5470, -0.0314, -0.2741, -0.0091,
          0.2288, -0.4567,  0.1581, -0.7147,  0.2120,  0.3984, -0.0408, -0.4238,
          0.5873,  0.5146, -0.1482, -0.2600,  0.1145,  0.5446,  0.4510,  0.7426],
        [-0.3996, -0.4326,  0.2144, -0.0050, -0.3459, -0.9060, -0.0207, -0.2460,
          0.0892,  0.5549,  0.2128,  0.2936, -0.5595, -0.0989, -0.2010, -0.1318,
          0.2966, -0.4456,  0.1263, -0.7435,  0.2342,  0.4642,  0.0160, -0.4048,
          0.6862,  0.6544, -0.0636, -0.2044,  0.1379,  0.4861,  0.3032,  0.7449],
        [-0.4490, -0.4387,  0.3479, -0.0325, -0.3824, -0.8228, -0.0367, -0.3237,
          0.0422,  0.5197,  0.3079,  0.2721, -0.6162, -0.1032, -0.2107,  0.0125,
          0.3507, -0.4331,  0.1407, -0.6841,  0.1594,  0.4995, -0.0403, -0.3999,
          0.6218,  0.6292, -0.0648, -0.2559,  0.1270,  0.5776,  0.3325,  0.8718],
        [-0.4317, -0.3606